# Scenarios

In [ ]:
import pandas as pd

dataframe = pd.read_csv("data/example.csv")
dataframe

In [ ]:
import atoti as tt

session = tt.create_session()
store = session.read_pandas(dataframe, "First", keys=["ID"])
cube = session.create_cube(store, "FirstCube")
cube.query()

## Scenarios from modified sources

Load a modified version of your data into the store to compare it to the original version.

First, let's modify the source dataset with _pandas_:

In [ ]:
dataframe.loc[(dataframe["City"] == "Paris"), "Color"] = "purple"
dataframe

We can now load this dataset into a new scenario:

In [ ]:
store.scenarios["Purple Paris"].load_pandas(dataframe)
store.scenarios["Purple Paris"]

If you want the store to appear in application's Source Simulation widget, you will need to enable this parameter on the store

In [ ]:
store.source_simulation_enabled = True

## Scenarios measure scaling

You can also perform simulations by modifying the value of certain measures. There are three available simulation methods:

- multiply: multiplies the measure's value with the provided weight
- replace: replaces the measure's value with the given value
- add: increments the measure's value with the provided value

In [ ]:
help(cube.setup_simulation)

In [ ]:
lvl = cube.levels
m = cube.measures

Calling the `cube.setup_simulation` method will create a special type of store which controls the simulation. This store's key fields are the levels you provide, as well as the column with the name of the simulation, which contains the scenario names.

In [ ]:
simulation = cube.setup_simulation(
    "First simulation",
    levels=[lvl["Continent"]],
    multiply=[m["Quantity.SUM"], m["Quantity.MEAN"]],
    replace=[m["Price.MEAN"]],
)
simulation

In [ ]:
simulation.head()

In order to populate this simulation with scenarios, you can either populate the simulation the same way you would a regular store, with a CSV file or a DataFrame. Or you can create a scenario for this simulation and then insert rows into it either manually or from a source file.

When filling a scenario, (not the simulation), there are several things which are done automatically. For instance, you don't need to specify the priority.

In [ ]:
asian_growth = simulation.scenarios["Asian growth"]
asian_growth += ("Asia", 1.05, 1.05, 20)
asian_growth += ("Europe", 0.9, 0.9, 0.85)
asian_growth.head()

Calling `head` on the scenario will display the rows of the Simulation handled by this scenario.

In [ ]:
cube.query(
    m["Quantity.SUM"],
    m["Quantity.MEAN"],
    m["Price.MEAN"],
    levels=[lvl["Continent"], lvl["First simulation"]],
)

You can create as many scenarios as you want. However, every scenario of a simulation uses the same methods. If you want to use a different method you must create a new simulation.

When loading a DataFrame into the simulation, you can get the required headers from the simulation. The same is true for a scenario. For the priority, you can use any numeric value you want. When loading a DataFrame into a scenario, the priority column is optional.

In [ ]:
growth = simulation.scenarios["Growth"]
normal_priority = 0
rows = [
    ["Asia", 1.1, 1.1, 22, normal_priority],
    ["Europe", 1.2, 1.3, 15, normal_priority],
]
df = pd.DataFrame(rows, columns=growth.columns)
df

In [ ]:
growth = simulation.scenarios["Growth"].load_pandas(df)
growth.head()

You can delete a simulation from the cube's simulation dict

In [ ]:
del cube.simulations["First simulation"]

You can create simulations wihout any levels. These simulations can only have one value per scenario. For simulations whithout any fields and acting on only one measure, you can assign values to scenarios instead of inserting tuples.

In [ ]:
no_field_sim = cube.setup_simulation("No Field Sim", multiply=[m["Price.SUM"]])
no_field_sim.scenarios["- 10 %"] = 0.9
no_field_sim.scenarios["+ 20%"] = 1.2
no_field_sim.head()

The priority rules are used to supersed the default behaviour between conflicting rules in the simulation. The basic rule is the more wildcard fields there are and the further to the left these wildcards are, the lower the priority. So in the following example, the rule on Europe overrides the one on France.
To use wildcards, simply pass `None` as the value for the column(s) you want the wildcard to be on.

In [ ]:
priority_sim = cube.setup_simulation(
    "Priority Simulation",
    levels=[lvl["Continent"], lvl["Country"]],
    multiply=[m["Quantity.MEAN"]],
)
priority_scenario = priority_sim.scenarios["growth"]
priority_scenario += ("Europe", None, 1.1)
priority_scenario += (None, "France", 1.2)
priority_scenario.head()

In [ ]:
cube.query(
    m["Quantity.MEAN"],
    levels=[lvl["Continent"], lvl["Country"], lvl["Priority Simulation"]],
)

To override the Europe rule with one specific to France, we can give a higher priority:

In [ ]:
priority_scenario += (
    None,
    "France",
    120,
    normal_priority + 1,
)
priority_scenario.head()

In [ ]:
cube.query(
    m["Quantity.MEAN"],
    levels=[lvl["Continent"], lvl["Country"], lvl["Priority Simulation"]],
)